In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import os
from PIL import Image
import numpy as np
import torch
from torch.utils.data import Dataset
from torchvision import transforms as T
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [2]:
VOC_ROOT = '/kaggle/input/pascal-voc-2012/VOC2012'  # PASCAL VOC数据集的根目录

In [3]:
class PascalVOCDataset(Dataset):
    def __init__(self, root, image_set='train', transform=None, target_transform=None):
        self.root = root
        self.image_set = image_set
        self.transform = transform
        self.target_transform = target_transform
        
        # 读取图像ID列表
        image_set_file = os.path.join(self.root, 'ImageSets/Segmentation', f'{self.image_set}.txt')
        with open(image_set_file, 'r') as f:
            self.images = [line.strip() for line in f.readlines()] # 将文件中的每一行都读出来，去掉换行符，然后存入一个 Python 列表中。如['2007_000032', '2007_000039', '2007_000063', '2007_000121', ...]
    
    def __len__(self):
        return len(self.images)
    
    def __getitem__(self, idx):
        # 获取图像和掩码路径
        image_id = self.images[idx]
        image_path = os.path.join(self.root, 'JPEGImages', f'{image_id}.jpg')
        mask_path = os.path.join(self.root, 'SegmentationClass', f'{image_id}.png')
        
        # 读取图像和掩码
        image = Image.open(image_path).convert('RGB')
        mask = Image.open(mask_path)
        
        # 应用变换
        if self.transform:
            image = self.transform(image)
        if self.target_transform:
            mask = self.target_transform(mask)
        
        # PASCAL VOC的掩码是调色板模式，不是RGB值，而是一个二维数组，需要转换为NumPy数组来获取类别索引
        mask = np.array(mask, dtype=np.int64)

        # 直接将NumPy数组转换为PyTorch张量
        mask = torch.from_numpy(mask)
        return image, mask

`mask = np.array(mask, dtype=np.int64)`
- 作用：PASCAL VOC的分割掩码（SegmentationClass里的.png文件）是一种特殊的“调色板模式”（Palette mode, 'P' mode）图像。你用肉眼看它是有各种颜色的，但它实际存储的不是RGB值，而是一个二维数组，数组里每个位置的值是该像素颜色在“调色板”中的索引。
- PASCAL VOC非常巧妙地将类别ID设置为了这个索引。所以，类别为0（背景）的像素，其索引值就是0；类别为1（飞机）的像素，其索引值就是1；...；而物体边界的特殊颜色，其索引值被设定为255。
- np.array(mask) 这行代码就是将这个PIL图像直接转换成一个包含这些索引值的NumPy二维数组。我们得到的 mask 就直接是一个 [H, W] 的矩阵，里面的值是 0, 1, 2, ..., 20 和 255。



问题一：既然是全卷积网络，为什么还要 `Resize`？

1. **批处理 (Batching) 的需要**
这是最主要、最根本的原因。为了高效利用GPU进行并行计算，我们不会一张一张地训练图像，而是将多张图像组成一个**批次 (batch)** 一起送入网络。

*   一个批次的数据在PyTorch中是一个单一的Tensor，例如 `(batch_size, channels, height, width)`，即 `(8, 3, 256, 256)`。
*   **一个Tensor中的所有元素必须具有相同的维度。** 你不能把一张 `256x256` 的图像和一张 `512x300` 的图像堆叠（stack）成一个批次。
*   因此，为了能将多张原始尺寸各不相同的图像打包成一个batch，我们必须在数据预处理阶段将它们`Resize`成一个统一的尺寸。

2. **内存管理与可预测性**
GPU的显存是有限的。
*   如果输入图像尺寸不固定，那么网络中间层产生的特征图尺寸也会变化，导致每一批次占用的显存大小也随之波动。
*   一张非常大的输入图像可能会直接导致显存溢出（Out of Memory, OOM），使训练崩溃。
*   通过`Resize`到一个固定的、中等大小的尺寸（如 `256x256` 或 `512x512`），我们可以精确地计算出模型需要的显存，并选择一个不会导致OOM的最大的`batch_size`，从而稳定、高效地进行训练。

3. **训练效率与稳定性**
*   处理大尺寸图像会消耗更多的计算资源，减慢训练速度。
*   将所有图像统一到一个中等尺寸，可以平衡细节保留和训练效率。对于PASCAL VOC这类数据集，`256x256`或`512x512`已经足够保留大部分物体的关键特征。

4. **转置卷积参数的固定**
我们之前在设计`FCN_AlexNet`时，上采样层`nn.ConvTranspose2d`的参数（`kernel_size`, `stride`, `padding`）是**为了将一个特定尺寸的特征图恢复到另一个特定尺寸而精心计算的**。

```python
# 对于 256x256 输入，下采样8倍后是 32x32
# kernel_size=16, stride=8, padding=4 可以将 32x32 恢复到 256x256
self.upsampler = nn.ConvTranspose2d(..., kernel_size=16, stride=8, padding=4)
```
如果输入尺寸变化，比如变成`257x257`，下采样8倍后会变成`32x32`（整数除法），但上采样恢复后尺寸可能就不再是`257x257`了，会导致输出和标签掩码尺寸不匹配。虽然FCN原文中提到了用裁剪（cropping）来解决这个问题，但这会增加实现的复杂性。在训练阶段固定输入尺寸，可以大大简化上采样模块的设计。

**总结**：`Resize`主要是为了**满足批处理的要求**，并带来**稳定的内存占用**和**高效的训练速度**。虽然牺牲了FCN处理任意尺寸输入的灵活性，但在训练阶段，这些工程上的好处远大于其带来的限制。在**推理（inference）阶段**，你完全可以加载训练好的模型，输入一张任意尺寸的图片进行分割。

---

问题二：`interpolation=transforms.InterpolationMode.NEAREST` 的作用是什么？

这个参数指定了在调整图像尺寸时使用的**插值算法**。插值算法决定了当一个像素点被映射到新尺寸图像的非整数坐标时，如何计算它的新像素值。

让我们用一个简单的例子来说明：

假设我们要把一个 `2x2` 的图像放大到 `4x4`。

**原始图像 (掩码)**
```
[[1, 2],
 [3, 4]]
```
这里的 `1, 2, 3, 4` 代表不同的物体类别（比如1=天空, 2=建筑, 3=道路, 4=汽车）。

1. **默认插值：双线性插值 (Bilinear Interpolation)**
这是`transforms.Resize`对普通RGB图像的默认方法。它会考虑周围4个像素点的值，并根据距离进行加权平均。

*   在新图像的某个位置，它可能会计算出 `(1+2)/2 = 1.5` 这样的值。
*   对于RGB图像，`1.5` 是一个有效的颜色值，它会让颜色过渡更平滑，看起来更自然。
*   但对于我们的**分割掩码**，`1.5` 是一个**灾难**！它既不是类别1（天空），也不是类别2（建筑），而是一个**不存在的、无意义的类别**。这会彻底搞乱我们的标签。

2. **最近邻插值 (Nearest Neighbor Interpolation)**
这就是 `interpolation=NEAREST` 的作用。它非常简单直接：**新位置的像素值，直接等于它在原始图像中最近的那个像素点的值。**

*   它不会创造任何新的值。输出图像中的所有像素值，都必然是原始图像中已经存在的值。
*   对于我们的掩码，这意味着放大后的掩码中，像素值只可能是 `1, 2, 3, 4` 中的一个。它完美地保留了类别的离散型和完整性。



In [4]:
IMAGE_SIZE= (256, 256)
BATCH_SIZE = 64
# 对图像的变换
image_transform = transforms.Compose([
    transforms.Resize(IMAGE_SIZE),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])
# 对掩码的变换
# 注意：对掩码只能进行尺寸变换，不能进行归一化等操作
# interpolation=NEAREST 保证缩放后不会产生新的类别值
mask_transform = transforms.Compose([
    transforms.Resize(IMAGE_SIZE, interpolation=transforms.InterpolationMode.NEAREST)
])

# 准备数据集和数据加载器
train_dataset = PascalVOCDataset(root=VOC_ROOT, image_set='train', transform=image_transform, target_transform=mask_transform)
val_dataset = PascalVOCDataset(root=VOC_ROOT, image_set='val', transform=image_transform, target_transform=mask_transform)
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=0)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=0)


In [5]:
# 构建网络模型
import torch.nn as nn
import torch.nn.functional as F


class FCN(nn.Module):
    def __init__(self, num_classes=21):
        super().__init__()
        
        # 256 -> 128 -> 64 -> 32 -> 16 -> 8
        
        # Block 1: 256 -> 128
        self.conv_block1 = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        
        # Block 2: 128 -> 64
        self.conv_block2 = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(128, 128, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        
        # Block 3: 64 -> 32
        self.conv_block3 = nn.Sequential(
            nn.Conv2d(128, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        
        # Block 4: 32 -> 16
        self.conv_block4 = nn.Sequential(
            nn.Conv2d(256, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        
        # Block 5: 16 -> 8
        self.conv_block5 = nn.Sequential(
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        
        # ==================== 分类器 (1x1 Conv) ====================
        self.classifier = nn.Conv2d(512, num_classes, kernel_size=1)
        
        # ==================== 上采样路径 ====================
        # 我们需要从 8x8 恢复到 256x256，需要上采样 32 倍
        self.upsample = nn.ConvTranspose2d(num_classes, num_classes, kernel_size=64, stride=32, padding=16) # (kernel_size - stride) / 2
    def forward(self, x):
        # 记录原始尺寸，以防万一
        input_size = x.shape[-2:]
        
        x = self.conv_block1(x)
        x = self.conv_block2(x)
        x = self.conv_block3(x)
        x = self.conv_block4(x)
        x = self.conv_block5(x)
        
        x = self.classifier(x)
        
        x = self.upsample(x)
        
        # 如果因为计算精度导致尺寸差一点点，可以用插值来修正
        x = F.interpolate(x, size=input_size, mode='bilinear', align_corners=False)
        
        return x




net = FCN(num_classes=21)
net = net.to(device)

In [6]:
import torch.optim as optim
# 损失函数
loss_function = nn.CrossEntropyLoss(ignore_index=255)  # PASCAL VOC中255是边界或忽略区域，使用 ignore_index=255 来忽略PASCAL VOC中的边界像素，边界像素(255)在计算损失时会被忽略
# 优化器
optimizer = optim.Adam(net.parameters(), lr=0.001)
# 学习率调度器
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, 
    mode='min',      # 监控的指标越小越好 (我们的目标是降低loss)
    factor=0.2,      # 当指标不再改善时，学习率乘以这个因子 (lr = lr * 0.2)
    patience=5,      # 容忍指标连续5个epoch不改善，才降低学习率
    verbose=True,    # 当学习率更新时，在控制台打印一条消息
    min_lr=1e-6      # 学习率的下限，防止降得太低
)

/usr/local/lib/python3.11/dist-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


In [7]:
from torch.utils.tensorboard import SummaryWriter
# --- 1. 初始化 ---
epochs = 500
writer = SummaryWriter("/kaggle/working/segmentation")
best_val_loss = float('inf') # 用于保存最佳模型的变量
MODEL_SAVE_PATH = '/kaggle/working/fcn_voc_best.pth'

print('Starting Training...')
# --- 2. 主训练循环 ---
for epoch in range(epochs): 
    print(f"--- Epoch {epoch + 1}/{epochs} ---")
    
    # --- 训练阶段 ---
    net.train() # 对于包含 Dropout 层或 Batch Normalization 层的模型需要调用 model.train() 和 model.eval() 
    train_loss = 0.0
    train_correct_pixels = 0
    train_total_pixels = 0
    
    # 使用tqdm包装训练数据加载器
    train_progress_bar = tqdm(train_loader, desc="Training", colour="green")
    
    for inputs, masks in train_progress_bar:
        inputs = inputs.to(device)
        masks  = masks.to(device, dtype=torch.long) # 确保掩码是长整型张量，因为 CrossEntropyLoss 需要标签是整数类型
        # 清除梯度->向前传播->计算损失->反向传播->优化参数
        optimizer.zero_grad() # zero_grad() 必须在 loss.backward()前. 在当前训练批次开始时，首先清零所有参数的梯度。
        outputs = net(inputs)
        loss = loss_function(outputs, masks)
        loss.backward()
        optimizer.step()

         # 累积损失
        train_loss += loss.item()

        # 计算像素准确率
        with torch.no_grad():
            preds = torch.argmax(outputs, dim=1)
            # 只计算非忽略区域的像素
            valid_pixels = (masks != 255)
            train_correct_pixels += ((preds == masks) & valid_pixels).sum().item()
            train_total_pixels += valid_pixels.sum().item()
        
        # 使用 set_postfix 在进度条上动态显示当前批次的损失
        train_progress_bar.set_postfix(loss=f"{loss.item():.4f}")

    # --- 验证阶段 ---
    net.eval()
    val_loss = 0.0
    val_correct_pixels = 0
    val_total_pixels = 0

    # 使用tqdm包装验证数据加载器
    val_progress_bar = tqdm(val_loader, desc="Validation", colour="red")
    
    with torch.no_grad(): # 测试过程中不计算梯度
        for inputs, masks in val_progress_bar:
            inputs = inputs.to(device)
            masks = masks.to(device, dtype=torch.long)
            outputs = net(inputs)
            loss = loss_function(outputs, masks)

            val_loss += loss.item()

            preds = torch.argmax(outputs, dim=1)
            valid_pixels = (masks != 255)
            val_correct_pixels += ((preds == masks) & valid_pixels).sum().item()
            val_total_pixels += valid_pixels.sum().item()

            val_progress_bar.set_postfix(loss=f"{loss.item():.4f}")


    # --- 3. 计算并记录每个Epoch的指标 ---
    # 计算平均损失和准确率
    avg_train_loss = train_loss / len(train_loader)
    train_pixel_acc = (train_correct_pixels / train_total_pixels) * 100 if train_total_pixels > 0 else 0
    
    avg_val_loss = val_loss / len(val_loader)
    val_pixel_acc = (val_correct_pixels / val_total_pixels) * 100 if val_total_pixels > 0 else 0
    # 打印本轮的最终结果
    print(f"\nEpoch {epoch + 1} Summary:")
    print(f"  Train Loss: {avg_train_loss:.4f}, Train Pixel Accuracy: {train_pixel_acc:.2f}%")
    print(f"  Validation Loss: {avg_val_loss:.4f}, Validation Pixel Accuracy: {val_pixel_acc:.2f}%")
    # 记录到TensorBoard
    writer.add_scalar('Loss/train', avg_train_loss, epoch)
    writer.add_scalar('Accuracy/train', train_pixel_acc, epoch)
    writer.add_scalar('Loss/validation', avg_val_loss, epoch)
    writer.add_scalar('Accuracy/validation', val_pixel_acc, epoch)
    writer.add_scalar('Learning_Rate', optimizer.param_groups[0]['lr'], epoch)
    
    # 更新学习率，并将验证集损失传入
    scheduler.step(avg_val_loss)
    print(f"  Current learning rate: {optimizer.param_groups[0]['lr']:.6f}")

    # --- 4. 保存最佳模型 ---
    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss
        torch.save(net.state_dict(), MODEL_SAVE_PATH)
        print(f"  ---> New best model saved to {MODEL_SAVE_PATH} (Validation Loss: {best_val_loss:.4f})")
    
    print("-" * 30)

print('Finished Training')
print(f"Best model saved at {MODEL_SAVE_PATH} with validation loss: {best_val_loss:.4f}")
writer.close()


2025-07-21 11:29:02.482211: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1753097342.673937      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1753097342.729851      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Starting Training...
--- Epoch 1/500 ---


Validation: 100%|██████████| 23/23 [00:40<00:00,  1.75s/it, loss=1.6080]



Epoch 1 Summary:
  Train Loss: 2.4954, Train Pixel Accuracy: 55.47%
  Validation Loss: 1.6013, Validation Pixel Accuracy: 73.29%
  Current learning rate: 0.001000
  ---> New best model saved to /kaggle/working/fcn_voc_best.pth (Validation Loss: 1.6013)
------------------------------
--- Epoch 2/500 ---


Validation: 100%|██████████| 23/23 [00:15<00:00,  1.46it/s, loss=1.3778]


...


Epoch 498 Summary:
  Train Loss: 0.3670, Train Pixel Accuracy: 88.64%
  Validation Loss: 1.3614, Validation Pixel Accuracy: 73.55%
  Current learning rate: 0.000001
------------------------------
--- Epoch 499/500 ---


Validation: 100%|██████████| 23/23 [00:14<00:00,  1.64it/s, loss=1.2872]



Epoch 499 Summary:
  Train Loss: 0.3669, Train Pixel Accuracy: 88.65%
  Validation Loss: 1.3613, Validation Pixel Accuracy: 73.56%
  Current learning rate: 0.000001
------------------------------
--- Epoch 500/500 ---


Validation: 100%|██████████| 23/23 [00:14<00:00,  1.55it/s, loss=1.2873]


Epoch 500 Summary:
  Train Loss: 0.3669, Train Pixel Accuracy: 88.65%
  Validation Loss: 1.3620, Validation Pixel Accuracy: 73.56%
  Current learning rate: 0.000001
------------------------------
Finished Training
Best model saved at /kaggle/working/fcn_voc_best.pth with validation loss: 0.9941


In [8]:

import matplotlib.pyplot as plt


MODEL_PATH = '/kaggle/working/fcn_voc_best.pth' # 你训练好的模型路径
OUTPUT_DIR = '/kaggle/working/results/'

NUM_CLASSES = 21
IMAGE_SIZE = (256, 256)
NUM_VISUALIZATIONS = 10

os.makedirs(OUTPUT_DIR, exist_ok=True)

# --- 1. PASCAL VOC 颜色映射 (不变) ---
VOC_COLORMAP = np.array([
    [0, 0, 0], [128, 0, 0], [0, 128, 0], [128, 128, 0],
    [0, 0, 128], [128, 0, 128], [0, 128, 128], [128, 128, 128],
    [64, 0, 0], [192, 0, 0], [64, 128, 0], [192, 128, 0],
    [64, 0, 128], [192, 0, 128], [64, 128, 128], [192, 128, 128],
    [0, 64, 0], [128, 64, 0], [0, 192, 0], [128, 192, 0],
    [0, 64, 128]
], dtype=np.uint8)

def mask_to_color(mask, colormap):
    color_mask = np.zeros((mask.shape[0], mask.shape[1], 3), dtype=np.uint8)
    for class_idx, color in enumerate(colormap):
        color_mask[mask == class_idx] = color
    return color_mask



# 定义数据变换
image_transform = transforms.Compose([
    transforms.Resize(IMAGE_SIZE),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])
mask_transform = transforms.Compose([
    transforms.Resize(IMAGE_SIZE, interpolation=transforms.InterpolationMode.NEAREST)
])
# 加载模型
model = FCN(num_classes=NUM_CLASSES).to(device)
model.load_state_dict(torch.load(MODEL_PATH, map_location=device))
model.eval()
print("模型加载成功！")
# 准备数据集
test_dataset = PascalVOCDataset(root=VOC_ROOT, image_set='trainval', transform=image_transform, target_transform=mask_transform)
# 注意：这里我们不使用DataLoader，因为我们需要文件名来加载原始图像，逐个处理更方便

print(f"开始在 '{test_dataset.image_set}' 集上进行预测和可视化...")

with torch.no_grad():
    # 我们直接遍历数据集的文件名列表
    for i in range(min(NUM_VISUALIZATIONS, len(test_dataset))):
        # 从数据集中获取处理好的图像和掩码
        image_tensor, gt_mask = test_dataset[i]
        image_id = test_dataset.images[i]
        # --- 这是关键的调整 ---
        # 手动加载原始图像用于可视化
        original_image_path = os.path.join(VOC_ROOT, 'JPEGImages', f'{image_id}.jpg')
        original_image = Image.open(original_image_path).convert('RGB').resize(IMAGE_SIZE)
        # ----------------------
        # 模型需要一个batch维度，所以我们用unsqueeze(0)增加一个维度
        image_tensor = image_tensor.unsqueeze(0).to(device)
        # 模型预测
        output = model(image_tensor)
        pred_mask = torch.argmax(output, dim=1)
        # 转为Numpy
        pred_mask_np = pred_mask.cpu().squeeze(0).numpy()
        gt_mask_np = gt_mask.numpy()
        # 将边界(255)处理一下，方便可视化
        gt_mask_np[gt_mask_np == 255] = 0 
        # 转为彩色图
        pred_mask_color = mask_to_color(pred_mask_np, VOC_COLORMAP)
        gt_mask_color = mask_to_color(gt_mask_np, VOC_COLORMAP)
        # 绘图
        fig, axes = plt.subplots(1, 3, figsize=(18, 6))
        axes[0].imshow(original_image)
        axes[0].set_title(f'Original Image: {image_id}')
        axes[0].axis('off')
        axes[1].imshow(gt_mask_color)
        axes[1].set_title('Ground Truth Mask')
        axes[1].axis('off')
        axes[2].imshow(pred_mask_color)
        axes[2].set_title('Predicted Mask')
        axes[2].axis('off')
        plt.tight_layout()
        
        save_path = os.path.join(OUTPUT_DIR, f'{image_id}_pred.png')
        plt.savefig(save_path)
        plt.close(fig)
        print(f"已保存可视化结果: {save_path}")
print(f"\n可视化完成！结果已保存在 '{OUTPUT_DIR}' 文件夹中。")


模型加载成功！
开始在 'trainval' 集上进行预测和可视化...
已保存可视化结果: /kaggle/working/results/2007_000032_pred.png
已保存可视化结果: /kaggle/working/results/2007_000033_pred.png
已保存可视化结果: /kaggle/working/results/2007_000039_pred.png
已保存可视化结果: /kaggle/working/results/2007_000042_pred.png
已保存可视化结果: /kaggle/working/results/2007_000061_pred.png
已保存可视化结果: /kaggle/working/results/2007_000063_pred.png
已保存可视化结果: /kaggle/working/results/2007_000068_pred.png
已保存可视化结果: /kaggle/working/results/2007_000121_pred.png
已保存可视化结果: /kaggle/working/results/2007_000123_pred.png
已保存可视化结果: /kaggle/working/results/2007_000129_pred.png

可视化完成！结果已保存在 '/kaggle/working/results/' 文件夹中。
